# Smartchain.bet

## A P2P Betting Blockchain-based Platform

### What is the bet lifecycle?

##### Events Created ==> OpenBet ==> AcceptBet ==> ResolveBet
#####
Events could terminate because one of 3 reasons:
- Resolve Bet
- Terminate Bet - for one bet
- Terminate all

Both "Terminate Bet" and "Terminate all" restitue the tokens to the e-wallet that had made the bet and depending on the state, as follows:

- if it is an open,restitute only to the BetInitiator
- If it is an accepted bet, restitute to both the BetInitiator and BetAcceptor

### 1. Events are created

In [1]:
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import os
import sys
# from crypto import convertDataToJSON, pinJSONtoIPFS, initContract, w3
from pathlib import Path
from web3 import Web3
# from web3.auto import w3
import numpy
import time

In [2]:
import abi

In [3]:
abi = abi.abi
abi

'[\n\t{\n\t\t"constant": false,\n\t\t"inputs": [\n\t\t\t{\n\t\t\t\t"name": "_event_id",\n\t\t\t\t"type": "uint256"\n\t\t\t},\n\t\t\t{\n\t\t\t\t"name": "_bet_id",\n\t\t\t\t"type": "uint256"\n\t\t\t},\n\t\t\t{\n\t\t\t\t"name": "_team_id",\n\t\t\t\t"type": "uint256"\n\t\t\t},\n\t\t\t{\n\t\t\t\t"name": "_bet_balance",\n\t\t\t\t"type": "uint256"\n\t\t\t}\n\t\t],\n\t\t"name": "acceptBet",\n\t\t"outputs": [],\n\t\t"payable": true,\n\t\t"stateMutability": "payable",\n\t\t"type": "function"\n\t},\n\t{\n\t\t"anonymous": false,\n\t\t"inputs": [\n\t\t\t{\n\t\t\t\t"indexed": true,\n\t\t\t\t"name": "event_id",\n\t\t\t\t"type": "uint256"\n\t\t\t},\n\t\t\t{\n\t\t\t\t"indexed": false,\n\t\t\t\t"name": "event_date",\n\t\t\t\t"type": "uint256"\n\t\t\t},\n\t\t\t{\n\t\t\t\t"indexed": false,\n\t\t\t\t"name": "teamH_id",\n\t\t\t\t"type": "uint256"\n\t\t\t},\n\t\t\t{\n\t\t\t\t"indexed": false,\n\t\t\t\t"name": "teamA_id",\n\t\t\t\t"type": "uint256"\n\t\t\t},\n\t\t\t{\n\t\t\t\t"indexed": false,\n\t\t\t\t"name"

In [11]:
def loadEvents(date_str):
    for idx in range(len(createBetsDF)):
        event_id = createBetsDF.iloc[idx]["GameID"].astype(int).item()
        event_date = createBetsDF.iloc[idx]["unixtimestamp"].astype(int).item()
        home_team = createBetsDF.iloc[idx]["HomeTeamID"].astype(int).item()
        away_team = createBetsDF.iloc[idx]["AwayTeamID"].astype(int).item()

        if (event_id not in available_events):
            nonce_var = w3.eth.get_transaction_count(authorized_wallet) ### Wallet Address from the account publishing the games
            betcreation_txn = contract.functions.publishEvent(event_id, event_date, home_team, away_team).buildTransaction({'from': authorized_wallet, 'value': 0,'gas': 12500000,'gasPrice': w3.toWei('5', 'gwei'),'nonce': nonce_var})
            signed_txn = w3.eth.account.sign_transaction(betcreation_txn, private_key=authorized_wallet_pk)
            result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
            print(f"result # {idx} - {result.hex()}")
            time.sleep(5)
            nonce_var += 1

In [12]:
load_dotenv(".env-dev")
gameDate = '12-Aug-2021' ##sys.argv[1]
sportsdataIO_key = os.getenv("FANTASY_DATA")
network_base_url = os.getenv("NETWORK_BASE_URL")
network_pk = os.getenv("NETWORK_PK")
smart_contract = os.getenv("SMART_CONTRACT")
authorized_wallet = os.getenv("AUTHORIZED_WALLET")
authorized_wallet_pk = os.getenv("AUTHORIZED_WALLET_PK")

gamesToday = requests.get(f"https://api.sportsdata.io/v3/mlb/scores/json/GamesByDate/{gameDate}?key=795e9822885d4100a28048652ad1bc8a")
gamesTodayDF = pd.DataFrame(gamesToday.json())

createBetsDF = gamesTodayDF[["GameID","DateTime","HomeTeamID","AwayTeamID"]]
createBetsDF["unixtimestamp"] = pd.to_datetime(createBetsDF["DateTime"]).astype(int)

/var/folders/5x/kls7bqyj73qbq2s9bq7cwn8m0000gn/T/ipykernel_40466/3690980551.py:14: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  createBetsDF["unixtimestamp"] = pd.to_datetime(createBetsDF["DateTime"]).astype(int)
/var/folders/5x/kls7bqyj73qbq2s9bq7cwn8m0000gn/T/ipykernel_40466/3690980551.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  createBetsDF["unixtimestamp"] = pd.to_datetime(createBetsDF["DateTime"]).astype(int)


In [13]:
createBetsDF

,GameID,DateTime,HomeTeamID,AwayTeamID,unixtimestamp
0,66682,2021-08-12T12:10:00,18,35,1628770200000000000
1,63081,2021-08-12T12:35:00,4,31,1628771700000000000
2,63077,2021-08-12T13:05:00,12,1,1628773500000000000
3,63079,2021-08-12T13:10:00,10,24,1628773800000000000
4,63078,2021-08-12T14:20:00,9,32,1628778000000000000
5,63085,2021-08-12T15:40:00,18,35,1628782800000000000
6,63076,2021-08-12T16:05:00,19,17,1628784300000000000
7,63082,2021-08-12T16:10:00,25,11,1628784600000000000
8,63083,2021-08-12T16:10:00,13,28,1628784600000000000
9,63074,2021-08-12T17:10:00,26,2,1628788200000000000


In [14]:
w3 = Web3(Web3.HTTPProvider(network_base_url+network_pk))
contract = w3.eth.contract(smart_contract, abi=abi)
nonce_var = w3.eth.get_transaction_count(authorized_wallet) ### Wallet Address from the account publishing the games
available_events = contract.functions.getAvailableEvents().call()

In this step, the company Smartchain.bet will run this script once a day in order to capture the events for the day. In this example we are only capturing MLB games using a demo API from sportsdata.io

The only parameter is the date for which the games you would want to load to the blockchain. Please note that this parameter is hardcoded here but in the running script is a parametes from the CLI to provide re-usability.

In [15]:
gameDate = '12-Aug-2021' ##sys.argv[1]

<b><i>loadEvents(gameDate)</i></b> is a function that takes the events from Sportsdata.io and submits them to the blockchain. Returns the hash of the transaction in the corresponding blockchain.

This function also validates the existance or not of the event in Python. This was a discovery...we originally had the validation in the Smart Contract, but then realized that even though it was working, the unintended consequences were that we had to paid a minimum amount of gas (due to the way the validation worked, it require to do some processing before checking). So we decided to do this in Python in order to avoid that cost.

In [38]:
loadEvents(gameDate)

result # 11 - 0xc61de808d8b4bfadaf1fa16b6d50120820e0c49a0d2f1fb4809981db68743209


if we go to the blockchain to check if the events were loaded correctly:

Let's first check the transaction #7 as an example:

In [39]:
createBetsDF.iloc[6]

GameID                         63076
DateTime         2021-08-12T16:05:00
HomeTeamID                        19
AwayTeamID                        17
unixtimestamp    1628784300000000000
Name: 6, dtype: object

Here we can see that the EventID is 63076 (event code provided by Sportsdata.io), the scheduled time of the game, who's local and who's away. Finally I converted the date from datetime to unix timestamp

But, that is in Python memory...let's check the blockchain:

In [41]:
available_events = contract.functions.getAvailableEvents().call()
print(f"Number of Events: {len(available_events)} = {available_events}")

Number of Events: 14 = [66682, 63079, 63081, 63077, 63076, 63083, 63074, 63779, 63075, 63078, 63082, 63080, 63085, 63084]


Above we can see that the events are already loaded in the Blockchain :) :) :) !!!!

### 2. Opening a bet

For the sake of the example, in this case I will "manually" enter the transactions through web3. The idea is to simulate what will happen with the front-end GUI.

Let's open a bet for 1000000000000 Weis, for the event 63076. Bet for team id 19 to win (timestamp is from the time I am running this)

In [42]:
nonce_var = w3.eth.get_transaction_count('0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5') ### Wallet Address from the account publishing the games
betcreation_txn = contract.functions.OpenBet(66682,1,1,18,1000000000000).buildTransaction({'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5', 
                                                                                          'value': 1000000000000,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                          'nonce': nonce_var})
betcreation_txn  

{'chainId': 42,
 'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5',
 'value': 1000000000000,
 'gas': 3000000,
 'gasPrice': 1000000000,
 'nonce': 761,
 'to': '0x5C6191D3168C9E337F621a94E41B6d8cad69851C',
 'data': '0xebf4718b000000000000000000000000000000000000000000000000000000000001047a000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000012000000000000000000000000000000000000000000000000000000e8d4a51000'}

bets before sending the transaction:

In [43]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Bets: {len(available_bets)} = {available_bets}")

Number of Bets: 0 = []


now we are signing it and send it to the blockchain

In [44]:
signed_txn = w3.eth.account.sign_transaction(betcreation_txn, private_key=authorized_wallet_pk)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")

result #0x2802d280d88150c425e22ded2ee871d549cb6fcac27246ff35cf7344a1823e58


Now we can check that the bets have incremented by 1

In [45]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Bets: {len(available_bets)} = {available_bets}")

Number of Bets: 1 = [1]


### 3. Accepting a bet

Now that we posted the bet, we change change wallets

In [46]:
authorized_wallet_2 = os.getenv("AUTHORIZED_WALLET_2")
authorized_wallet_pk_2 = os.getenv("AUTHORIZED_WALLET_PK_2")

318


In [55]:
nonce_var = w3.eth.get_transaction_count('0x1c322400C203d634928541a83cf702263e62472A') ### Wallet Address from the account accepting the bet
betacceptance_txn = contract.functions.acceptBet(66682,1,16,1000000000000).buildTransaction({'from': '0x1c322400C203d634928541a83cf702263e62472A', 
                                                                                          'value': 1000000000000,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                          'nonce': nonce_var})
betacceptance_txn  

{'chainId': 42,
 'from': '0x1c322400C203d634928541a83cf702263e62472A',
 'value': 1000000000000,
 'gas': 3000000,
 'gasPrice': 1000000000,
 'nonce': 319,
 'to': '0x5C6191D3168C9E337F621a94E41B6d8cad69851C',
 'data': '0x9440e63e000000000000000000000000000000000000000000000000000000000001047a00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000e8d4a51000'}

In [56]:
matched_bets = contract.functions.getMatchedBets().call()
print(f"Number of Bets: {len(matched_bets)} = {matched_bets}")

Number of Bets: 0 = []


In [57]:
signed_txn = w3.eth.account.sign_transaction(betacceptance_txn, private_key=authorized_wallet_pk_2)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")

result #0x4832898470f6a13722a150df1afdb1eb244b0d204d03c09ca37e883dc4383472


In [59]:
matched_bets = contract.functions.getMatchedBets().call()
print(f"Number of Bets: {len(matched_bets)} = {matched_bets}")

Number of Bets: 1 = [1]


### 4. Resolving a bet

Now that the bet has been accepted, let's assume that the game has happened and that the betInitiator won.

Then, we have the method "resolveBet" that executes this process.

The way this works is that the winner gets transferred the amount of the wage to her/his wallet.

Note: this function has a require that only allowed wallets can access and execute it

In [60]:
authorized_wallet_smart = os.getenv("AUTHORIZED_WALLET_SMARTCHAIN")
authorized_wallet_pk_smart = os.getenv("AUTHORIZED_WALLET_PK_SMARTCHAIN")

Checking for # of Resolved Bets (to show one more was added)

In [62]:
resolved_bets = contract.functions.getResolvedBets().call()
print(f"Number of Bets: {len(resolved_bets)} = {resolved_bets}")

Number of Bets: 0 = []


Create the transaction:

In [70]:
nonce_var = w3.eth.get_transaction_count('0x1c322400C203d634928541a83cf702263e62472A')
resolvebet_txn = contract.functions.resolveBet(63076,1,False).buildTransaction({'from': '0x1c322400C203d634928541a83cf702263e62472A', 
                                                                                          'value': 1000000000000,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                          'nonce': nonce_var})
resolvebet_txn  

{'chainId': 42,
 'from': '0x1c322400C203d634928541a83cf702263e62472A',
 'value': 1000000000000,
 'gas': 3000000,
 'gasPrice': 1000000000,
 'nonce': 321,
 'to': '0x5C6191D3168C9E337F621a94E41B6d8cad69851C',
 'data': '0x28635df0000000000000000000000000000000000000000000000000000000000000f66400000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000'}

In [71]:
signed_txn = w3.eth.account.sign_transaction(resolvebet_txn, private_key=authorized_wallet_pk_2)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")

result #0x0b673ac479969a14f34aaa52577da9d157276b12b8cdafb2f09a5456103a1834


In [74]:
resolved_bets = contract.functions.getResolvedBets().call()
print(f"Number of Resolved Bets: {len(resolved_bets)} = {resolved_bets}")

Number of Resolved Bets: 1 = [1]


Bet 1 has been resolved!!!!

Now the status of the Bets is:

In [75]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Available Bets: {len(available_bets)} = {available_bets}")

matched_bets = contract.functions.getMatchedBets().call()
print(f"Number of Matched Bets: {len(matched_bets)} = {matched_bets}")

resolved_bets = contract.functions.getResolvedBets().call()
print(f"Number of Resolved Bets: {len(resolved_bets)} = {resolved_bets}")

Number of Available Bets: 0 = []
Number of Matched Bets: 0 = []
Number of Resolved Bets: 1 = [1]


## Special Use Cases

### 4. Terminating a Bet

In [78]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Available Bets: {len(available_bets)} = {available_bets}")

Number of Available Bets: 1 = [2]


Let's add a Bet to demonstrate this functionality

In [76]:
nonce_var = w3.eth.get_transaction_count('0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5')
betcreation_txn = contract.functions.OpenBet(63076,1,2,19,1000000000000).buildTransaction({'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5', 
                                                                                          'value': 1000000000000,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                          'nonce': nonce_var})
signed_txn = w3.eth.account.sign_transaction(betcreation_txn, private_key=authorized_wallet_pk)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")

result #0xf783a6d0ce73b175f5d8dbd7b0448ea65f5df750bd805c0dfd618b5f6ae23ecb


In [79]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Available Bets: {len(available_bets)} = {available_bets}")

Number of Available Bets: 1 = [2]


Let's terminate Bet #2

In [89]:
nonce_var = w3.eth.get_transaction_count('0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5') ### Wallet Address from the account publishing the games
terminatebet_txn = contract.functions.terminateOne(2).buildTransaction({'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5', 
                                                                                  'value': 0,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                  'nonce': nonce_var})
terminatebet_txn  

{'chainId': 42,
 'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5',
 'value': 0,
 'gas': 3000000,
 'gasPrice': 1000000000,
 'nonce': 763,
 'to': '0x5C6191D3168C9E337F621a94E41B6d8cad69851C',
 'data': '0xa5cf1bd00000000000000000000000000000000000000000000000000000000000000002'}

In [91]:
signed_txn = w3.eth.account.sign_transaction(terminatebet_txn, private_key=authorized_wallet_pk)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")

result #0x9651ac1d0453dd6f14dd2774bb001c0a7ac1024230270933eb79712b9fa9604f


In [93]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Available Bets: {len(available_bets)} = {available_bets}")

Number of Available Bets: 0 = []


### 5. Terminating a All Bets

In [94]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Available Bets: {len(available_bets)} = {available_bets}")

Number of Available Bets: 0 = []


Let's add some bets in order to demonstrate the functionality

In [95]:
nonce_var = w3.eth.get_transaction_count('0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5')
betcreation_txn = contract.functions.OpenBet(63076,5,3,19,1000000000000).buildTransaction({'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5', 
                                                                                          'value': 1000000000000,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                          'nonce': nonce_var})
signed_txn = w3.eth.account.sign_transaction(betcreation_txn, private_key=authorized_wallet_pk)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")

result #0xa27cf90fcf89459374aa88ab29995e314c6bcd9b5ca7f824766aac1f5c63217c


In [97]:
nonce_var = w3.eth.get_transaction_count('0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5')
betcreation_txn = contract.functions.OpenBet(63076,5,4,19,1000000000000).buildTransaction({'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5', 
                                                                                          'value': 1000000000000,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                          'nonce': nonce_var})
signed_txn = w3.eth.account.sign_transaction(betcreation_txn, private_key=authorized_wallet_pk)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")
time.sleep(2)

result #0x620ec8c96e9eeec60b4d8e2bd5a9c9dc4d1b4f113107e16be24727a9ab375865


In [98]:
nonce_var = w3.eth.get_transaction_count('0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5')
betcreation_txn = contract.functions.OpenBet(63076,5,5,10,1000000000000).buildTransaction({'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5', 
                                                                                          'value': 1000000000000,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                          'nonce': nonce_var})
signed_txn = w3.eth.account.sign_transaction(betcreation_txn, private_key=authorized_wallet_pk)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")
time.sleep(2)

result #0x5e1ba33588cd4ca6f40eec6b5ccad5d8d80b6992fd907a64a61838e759fc9118


In [101]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Available Bets: {len(available_bets)} = {available_bets}")

Number of Available Bets: 3 = [3, 4, 5]


Now let's call the terminateAll method from the smart contract:

In [102]:
nonce_var = w3.eth.get_transaction_count('0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5')
terminateallbets_txn = contract.functions.terminateAll().buildTransaction({'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5', 
                                                                                  'value': 0,'gas': 3000000,'gasPrice': w3.toWei('1', 'gwei'),
                                                                                  'nonce': nonce_var})
terminateallbets_txn  

{'chainId': 42,
 'from': '0xDaf36E4570e2f0A587331b8E1E3645Ce8861B6A5',
 'value': 0,
 'gas': 3000000,
 'gasPrice': 1000000000,
 'nonce': 768,
 'to': '0x5C6191D3168C9E337F621a94E41B6d8cad69851C',
 'data': '0x4f984c5c'}

In [104]:
signed_txn = w3.eth.account.sign_transaction(terminateallbets_txn, private_key=authorized_wallet_pk)
result = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
print(f"result #{result.hex()}")

result #0x13d1bd9302efb027f0e7f7b8654a278f8163454e847497dc5b7fca95f9712873


Then we can see that all bets were eliminated

In [107]:
available_bets = contract.functions.getAvailableBets().call()
print(f"Number of Available Bets: {len(available_bets)} = {available_bets}")

Number of Available Bets: 0 = []
